In [1]:
import json
from pymatgen import Structure
from fireworks import Workflow
from atomate.vasp.fireworks.core import StaticFW
from pymatgen.io.vasp.sets import MPHSEBSSet
from atomate.vasp.powerups import add_modify_incar, add_additional_fields_to_taskdocs,\
add_small_gap_multiply, add_priority
from fireworks import LaunchPad

In [2]:
# Connect to launchpad (need to input details for your mongoDB
launchpad = LaunchPad(host="<<HOST>>", port=12345,
                      username="<<USER>>", password="<<PASSWORD>>", 
                      name="<<DB-NAME>>")

In [ ]:
# Import structrures from json
with open('data/example_input_structures.json', 'r') as f:
    structures = json.load(f)
    
# Convert to to pymatgen Structure objects
structures = [{'structure': Structure.from_dict(i['structure']), 'prototype_number': i['prototype_number']} 
                             for i in structures]

print(len(structures))

In [105]:
# We define our own workflow, modifying the existing StaticFW firework
# and using the MPHSEBSSet as a vasp input set. 
# We are using kpoints reciprocal_density of 120, 20% more than used for MP static calcs
# (which in turn are 50% more than are used for MP relaxations)
def get_HSE_singleshot_wf(structure, name="HSE gap wf", vasp_input_set=None,
                    vasp_cmd=">>vasp_cmd<<", db_file=">>db_file<<", user_kpoints_settings=None,
                    tag="", metadata=None):
    """
    Returns a simple HSE gap workflow.

    Args:
        structure (Structure): input structure to be optimized and run
        name (str): some appropriate name for the transmuter fireworks.
        vasp_input_set (DictSet): vasp input set.
        vasp_cmd (str): command to run
        db_file (str): path to file containing the database credentials.
        user_kpoints_settings (dict): example: {"grid_density": 7000}
        tag (str): some unique string that will be appended to the names of the fireworks so that
            the data from those tagged fireworks can be queried later during the analysis.
        metadata (dict): meta data

    Returns:
        Workflow
    """
    # input set for calculation
    vis_default = vasp_input_set or MPHSEBSSet(structure, mode='uniform', 
                                               user_kpoints_settings={"reciprocal_density": 120},
                                              copy_chgcar=False)
    if user_kpoints_settings:
        v = vis_default.as_dict()
        v.update({"user_kpoints_settings": user_kpoints_settings})
        vis_default = vis_default.__class__.from_dict(v)

    # Static firework
    fws = [StaticFW(structure=structure, vasp_input_set=vis_default, vasp_cmd=vasp_cmd,
                      db_file=db_file, name="{} hybrid single shot".format(tag))]

    wfname = "{}:{}".format(structure.composition.reduced_formula, name)

    return Workflow(fws, name=wfname, metadata=metadata)

In [106]:
for entry in structures:
    struct = entry['structure']
    
    # Suss out f orbitals
    comp = struct.composition
    atomic_numbers = [i.number for i in comp]
    if max(atomic_numbers) > 56:
        HFLMAX = 6
    else:
        HFLMAX = 4
    
    orig_wf = get_HSE_singleshot_wf(struct)
    modified_wf = add_modify_incar(orig_wf, modify_incar_params={'incar_update':
                                                                 { 'NWRITE': 2, 'ISTART': 1, 'INIWAV': 1,
                                                                 'NCORE': 8, 'KPAR': 4, 'HFSCREEN':0.207,
                                                                 'PREC': 'Accurate', 'ALGO': 'All',
                                                                  'ENCUT': 550,
                                                                  'EDIFF': 1E-6, 'GGA': 'PE',
                                                                  'LASPH': '.TRUE.', 'LREAL': 'Auto',
                                                                  'ADDGRID': '.TRUE.',
                                                                  'ISYM': 3,
                                                                  'ISMEAR': 0, 'SIGMA': 0.05,
                                                                  'LORBIT': 11, 'NEDOS': 3000,
                                                                  'HFLMAX': HFLMAX
                                                                   }})
    
    
    modified_wf = add_priority(modified_wf, 1000)
    launchpad.add_wf(modified_wf)

2019-06-05 12:16:35,934 INFO Added a workflow. id_map: {-29: 48}
2019-06-05 12:16:35,941 INFO Added a workflow. id_map: {-30: 49}
2019-06-05 12:16:35,952 INFO Added a workflow. id_map: {-31: 50}
2019-06-05 12:16:35,965 INFO Added a workflow. id_map: {-32: 51}
2019-06-05 12:16:35,971 INFO Added a workflow. id_map: {-33: 52}
